# Homework 2
Sam Friedman

## Problem 1
Find d-separated nodes. Assume that we have a Bayesian network graph $\mathit{G}$. Given a set of observed nodes $\mathbf{E}$, we want to find the set of nodes $\mathit{Y}$ that contains all nodes that are d-separated from the source node $X$.
    
    

In [158]:
# Create a Graph object
class Graph(object):
    ''' Graph object that has sets of nodes and edges '''
    def __init__(self, edges=set(), nodes=set(), observations=set()):
        self.parents = {}
        self.children = {}
        self.nodes = nodes
        if len(nodes) > 0:
            self.size = len(nodes)
        else:
            self.size = 0
        self.edges = edges
        self.obs = observations

        self.build_parent_and_child_sets()
    
    def add_node(self, node):
        ''' Add node to graph '''
        if node not in self.nodes:
            self.size += 1
            self.nodes.add((node))
        else:
            pass
    
    def build_parent_and_child_sets(self):
        for edge in self.edges:
            u, v = edge
            self.add_node(u)
            self.add_node(v)
                
            if v not in self.parents:
                self.parents.update({v : set(u)})
            else:
                self.parents[v].add(u)

            if u not in self.children:
                self.children.update({u : set(v)})
            else:
                self.children[u].add(v)
                
    def get_parents(self, node):
        ''' Return parents of node '''
        if node in self.parents:
            return self.parents[node]
        else:
            return set()
    
    def get_children(self, node):
        ''' Return children of node '''
        if node in self.children:
            return self.children[node]
        else:
            return set()
                
    def find_ancestors(self, source_node):
        ''' Use a breadth-first search to find all ancestors of node '''
        visited = set()  # initialize ancestor set
        queue = [source_node]              
        while queue:
            vertex = queue.pop(0)
            if vertex not in visited:
                visited.add(vertex)
                queue.extend(self.get_parents(vertex)-visited)
        return (visited - set(source_node))
    
    def find_descendants(self, source_node):
        ''' Use a breadth-first search to find all descendants of node '''
        visited = set()  # initialize descendant set
        queue = [source_node]              
        while queue:
            vertex = queue.pop(0)
            if vertex not in visited:
                visited.add(vertex)
                queue.extend(self.get_children(vertex)-visited)
        return (visited - set(source_node))
        
                
    def d_separated(self, X, Z):
        ''' 
        Find all nodes that are d_separated from node given observation
        Based on Algorithm 3.1, page 75, in PGM book
        
        X: starting node
        Z: set of observations
        
        '''
        # Phase 1: insert all ancestors of Z into A
        L = Z.copy()  # set L to be the set of observations
        A = set()  # set A to be the empty set
#         print('%% A={}'.format(A),'  L={}'.format(L))
        while len(L) != 0:
            Y = L.pop()  # remove an item from the set
#             print('!! A={}'.format(A),'  Y={}'.format(Y),'  L={}'.format(L))
            if Y not in A:
                L = L.union(self.get_parents(Y))  # Y's parents need to be visited
            A = A.union(Y)  # Y is ancestor of evidence
        print('$$ A={}'.format(A),'  Y={}'.format(Y),'  L={}'.format(L))
            
        # Phase 2: traverse active trails starting from node
        L = set([(X, 'up')])
        V = set()  # (node, direction) marked as visited
        R = set()  # Nodes reachable via active trail
        while len(L) != 0:
            print('hello!!')
            # select some (Y, dr) from L
            (Y, dr) = L.pop()
            print('(Y,dr)={}'.format((Y,dr)))
            if (Y, dr) not in V:
                print('&& R={}'.format(R),'  Y={}'.format(Y),'  Z={}'.format(Z),'  L={}'.format(L))
                if Y not in Z:
                    R = R.union(Y)  # Y is reachable
                    print('RRR={}'.format(R))
                V = V.union(set([(Y,dr)]))  # mark (Y, dr) as visited
                print('V={}'.format(V),'  Y={}'.format(Y),'  L={}'.format(L),'  Z={}'.format(Z))
                print('dr={}'.format(dr))
                if dr=='up' and Y not in Z:
                    # trail up through Y active if Y not in Z
                    print('yoyoyo')
                    print('parents of {} -> {}'.format(Y,self.get_parents(Y)))
                    tmp_iter= Z.intersection(self.get_parents(Y))
                    print('tmp_iter={}'.format(tmp_iter))
                    for z in tmp_iter:
                        print('z={}'.format(z))
                        L = L.union(set([(z,'up')])) # Y's parents to be visited from bottom
                    print('children of {} -> {}'.format(Y,self.get_children(Y)))
                    tmp_iter= Z.intersection(self.get_children(Y))
                    print('**tmp_iter={}'.format(tmp_iter))
                    for z in (Z.intersection(self.get_children(Y))):
                        L = L.union(set([(z,'down')])) # Y's children to be visited from top
                        print('z={}'.format(z),' L={}'.format(L))
                elif dr=='down':
                    # trails down through Y
                    if Y not in Z:
                        # downward trails to Y's children are active
                        for z in (Z.intersection(self.get_children(Y))):
                            print('z={}'.format(z))
                            L = L.union(set([(z,'down')]))  # Y's children to be visited from top
                    if Y in A:
                        # v-structure trails are active
                        for z in (Z.intersection(self.get_parents(Y))):
                            print('z={}'.format(z))
                            L = L.union(set([(z,'up')])) # Y's parents to be visited from bottom
        return R
                        

# Build Graph from homework 1, problem 3
nodes = set(('A','B','C','D','E','F','G','H','I','J','K','L','M'))
edges = set([('A','D'),('B','D'),('D','G'),('D','H'),('G','K'),('H','K'),\
             ('H','E'),('C','E'),('E','I'),('F','I'),('F','J'),('I','L'),\
             ('J','M')])
G = Graph(edges=edges)

In [159]:
# part (f)
Z = set(('K','E'))
dsep_nodes = G.d_separated('H', Z)
print('dsep nodes:', dsep_nodes)

$$ A={'K', 'E', 'B', 'A', 'H', 'G', 'D', 'C'}   Y=D   L=set()
hello!!
(Y,dr)=('H', 'up')
&& R=set()   Y=H   Z={'K', 'E'}   L=set()
RRR={'H'}
V={('H', 'up')}   Y=H   L=set()   Z={'K', 'E'}
dr=up
yoyoyo
parents of H -> {'D'}
tmp_iter=set()
children of H -> {'K', 'E'}
**tmp_iter={'K', 'E'}
z=K  L={('K', 'down')}
z=E  L={('K', 'down'), ('E', 'down')}
hello!!
(Y,dr)=('K', 'down')
&& R={'H'}   Y=K   Z={'K', 'E'}   L={('E', 'down')}
V={('K', 'down'), ('H', 'up')}   Y=K   L={('E', 'down')}   Z={'K', 'E'}
dr=down
hello!!
(Y,dr)=('E', 'down')
&& R={'H'}   Y=E   Z={'K', 'E'}   L=set()
V={('K', 'down'), ('E', 'down'), ('H', 'up')}   Y=E   L=set()   Z={'K', 'E'}
dr=down
dsep nodes: {'H'}


In [72]:

# # print('children', G.children)
# # print('parents', G.parents)
# # print('nodes', G.nodes)
# # print('size', G.size)

# start = 'L'
# visited, stack = set(), [start]
# while stack:
#     vertex = stack.pop()
#     if vertex not in visited:
#         visited.add(vertex)
#         stack.extend(G.get_parents(vertex) - visited)
# result = visited-set((start))
# print('end visited {}'.format(result))
# a1 = result
# a2 = G.find_ancestors('L')
# print('a1',a1)
# print('a2',a2)
# print(a1 == a2)

# node = 'I'
# print('find desc of',node)
# print(G.find_descendants(node))

# # find ancestors
# print('find ancestors')
# print(G.find_ancestors('L'))
# print('--find ancestors')



# nodes = set(('A','B','C','D','E','F','G','H','I','J','K','L','M'))
# print(nodes)

dsep nodes: None


In [102]:
A = set()
i = 0
print('hello')
Z = set(('K','E'))
Z.pop()
b = set(('asdf','asdf','qwe','g'))
a=Z.union(b)
print(Z)
print(a)
node = 'L'
L = set([(node, 'up'),('B','down')])
L
('B', 'up') in L

hello
{'E'}
{'qwe', 'asdf', 'g', 'E'}


False

In [41]:
s = set()
s.add('a')
print(s)
s.add('b')
print(s)
s.add('a')
s

l = []
l.append('b')
print(l)
l2 = set(l)
print(l2)
print(isinstance(l2, list))
isinstance(l2, set)

print(l2)
l2.add({})
print(l2)

{'a'}
{'b', 'a'}
['b']
{'b'}
False
{'b'}


TypeError: unhashable type: 'dict'